# Arch and Freq Invariance

This notebook shows the architecture and cpu frequency invariance features of EAS.

In [5]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')

# Enable logging at INFO level
logging.getLogger().setLevel(logging.INFO)
# Comment the follwing line to disable devlib debugging statements
# logging.getLogger('ssh').setLevel(logging.DEBUG)

In [6]:
# Generate plots inline
%pylab inline

import json
import os

import subprocess

# Support to access the remote target
import devlib
from env import TestEnv

# Support for trace events analysis
from trace import Trace
#from trace_analysis import TraceAnalysis

# Support to configure and run RTApp based workloads
from wlgen import RTA

# Support for performance analysis of RTApp workloads
from perf_analysis import PerfAnalysis

# Suport for FTrace events parsing and visualization
import trappy

Populating the interactive namespace from numpy and matplotlib


# Test environment setup

In [7]:
# Setup a target configuration
my_target_conf = {
    
    # Define the kind of target platform to use for the experiments
    "platform"    : 'android',  # Linux system, valid other options are:
                              # android - access via ADB
                              # linux   - access via SSH
                              # host    - direct access
    'ANDROID_HOME' : '/media/build/workspace/android-sdk',

    # Preload settings for a specific target
    "board"       : 'juno',   # load JUNO specific settings, e.g.
                              # - HWMON based energy sampling
                              # - Juno energy model
                              # valid options are:
                              # - juno  - JUNO Development Board
                              # - tc2   - TC2 Development Board
                              # - oak   - Mediatek MT63xx based target

    # Define devlib module to load
    #"modules"     : [
    #    'bl',           # enable big.LITTLE support
    #    'cpufreq'       # enable CPUFreq support
    #],

    # Account to access the remote target
    #"host"        : '192.168.1.101',
    "host"        : '10.169.36.74',
    "username"    : 'root',
    "password"    : '',

    # Comment the following line to force rt-app calibration on your target
    "rtapp-calib" : {
        '0': 361, '1': 138, '2': 138, '3': 352, '4': 360, '5': 353
    }

}

# Setup the required Test Environment supports
my_tests_conf = {
    
    # Binary tools required to run this experiment
    # These tools must be present in the tools/ folder for the architecture
    "tools"   : ['rt-app', 'taskset', 'trace-cmd'],
    
    # FTrace events end buffer configuration
    "ftrace"  : {
         "events" : [
            "cpu_frequency",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_switch",
            'dequeue_task_fair', 
            'enqueue_task_fair', 
            'set_next_entity',

                    
            "cpu_capacity",
            
            #/sys/kernel/debug/tracing/events/sched
            "sched_blocked_reason",
            "sched_boost_cpu",
            "sched_boost_task",
            "sched_contrib_scale_f",
            "sched_cpu_hotplug",
            "sched_energy_diff",
            "sched_kthread_stop",
            "sched_kthread_stop_ret",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_migrate_task",
            "sched_move_numa",
            "sched_pi_setprio",
            "sched_process_exec",
            "sched_process_exit",
            "sched_process_fork",
            "sched_process_free",
            "sched_process_wait",
            "sched_stat_blocked",
            "sched_stat_iowait",
            "sched_stat_runtime",
            "sched_stat_sleep",
            "sched_stat_wait",
            "sched_stick_numa",
            "sched_swap_numa",
            "sched_switch",
            "sched_tune_boostgroup_update",
            "sched_tune_config",
            "sched_tune_tasks_update",
            "sched_tune_filter",
            "sched_wait_task",
            "sched_wake_idle_without_ipi",
            "sched_wakeup",
            "sched_wakeup_new",
         ],
         "buffsize" : 10240
    },

    "results_dir" : "arch_and_freq_invariance",
}

In [8]:
# Support to access the remote target
import devlib
from env import TestEnv

# Initialize a test environment using:
# the provided target configuration (my_target_conf)
# the provided test configuration   (my_test_conf)
te = TestEnv(target_conf=my_target_conf, test_conf=my_tests_conf)
target = te.target

04:20:01  INFO    : Using base path: /media/build/workspace/lisa
04:20:01  INFO    : Loading custom (inline) target configuration
04:20:01  INFO    : Loading custom (inline) test configuration
04:20:01  INFO    : External tools using:
04:20:01  INFO    :    ANDROID_HOME: /media/build/workspace/android-sdk
04:20:01  INFO    :    CATAPULT_HOME: /media/build/workspace/lisa/tools/catapult
04:20:01  INFO    : Devlib modules to load: ['bl', 'cpufreq', 'hwmon']
04:20:01  INFO    : Connecting Android target [10.169.36.74:5555]
04:20:01  INFO    : Connection settings:
04:20:01  INFO    :    {'device': '10.169.36.74:5555'}


adb connect 10.169.36.74:5555


04:20:03  INFO    : Initializing target workdir:
04:20:03  INFO    :    /data/local/tmp/devlib-target
04:20:06  INFO    : Topology:
04:20:06  INFO    :    [[0, 3, 4, 5], [1, 2]]
04:20:07  INFO    : Loading default EM:
04:20:07  INFO    :    /media/build/workspace/lisa/libs/utils/platforms/juno.json
04:20:07  WARNING : Event [dequeue_task_fair] not available for tracing
04:20:07  WARNING : Event [enqueue_task_fair] not available for tracing
04:20:07  WARNING : Event [set_next_entity] not available for tracing
04:20:07  INFO    : Enabled tracepoints:
04:20:07  INFO    :    cpu_frequency
04:20:07  INFO    :    sched_load_avg_cpu
04:20:07  INFO    :    sched_load_avg_task
04:20:07  INFO    :    sched_switch
04:20:07  INFO    :    dequeue_task_fair
04:20:07  INFO    :    enqueue_task_fair
04:20:07  INFO    :    set_next_entity
04:20:07  INFO    :    cpu_capacity
04:20:07  INFO    :    sched_blocked_reason
04:20:07  INFO    :    sched_boost_cpu
04:20:07  INFO    :    sched_boost_task
04:20:0

# Workload configuration

In [9]:
# Support to configure and run RTApp based workloads
from wlgen import RTA, Periodic, Ramp

# Create a new RTApp workload generator using the calibration values
# reported by the TestEnv module
rtapp = RTA(target, 'simple', calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(
    # 1. generate a "profile based" set of tasks
    kind='profile',
    
    # 2. define the "profile" of each task
    params={
        
        
        'task_p50': Periodic(
            period_ms=400,         # period
            duty_cycle_pct=50,     # duty cycle
            duration_s=10,         # duration    
            cpus=str(target.bl.bigs[0])   # pinned on first big CPU
        ).get(),
        
        'task_r20': Ramp(
            start_pct=5,           # intial load
            end_pct=95,            # end load
            delta_pct=10,          # load % increase...
            time_s=1,              # ... every 1[s]            
        ).get(),
    },
    
    # 4. use this folder for task logfiles
    run_dir=target.working_directory
    
);

04:20:07  INFO    : Setup new workload simple
04:20:07  INFO    : Workload duration defined by longest task
04:20:07  INFO    : Default policy: SCHED_OTHER
04:20:07  INFO    : ------------------------
04:20:07  INFO    : task [task_p50], sched: using default policy
04:20:07  INFO    :  | calibration CPU: 1
04:20:07  INFO    :  | loops count: 1
04:20:07  INFO    :  | CPUs affinity: 1
04:20:07  INFO    : + phase_000001: duration 10.000000 [s] (25 loops)
04:20:07  INFO    : |  period   400000 [us], duty_cycle  50 %
04:20:07  INFO    : |  run_time 200000 [us], sleep_time 200000 [us]
04:20:07  INFO    : ------------------------
04:20:07  INFO    : task [task_r20], sched: using default policy
04:20:07  INFO    :  | calibration CPU: 1
04:20:07  INFO    :  | loops count: 1
04:20:07  INFO    : + phase_000001: duration 1.000000 [s] (10 loops)
04:20:07  INFO    : |  period   100000 [us], duty_cycle   5 %
04:20:07  INFO    : |  run_time   5000 [us], sleep_time  95000 [us]
04:20:08  INFO    : + pha

# Workload execution round 1

In [10]:
# Set performance governor
#
# For Juno big/LITTLE CPUs
#
logging.info("Target ABI: %s, CPus: %s",
             target.abi,
             target.cpuinfo.cpu_names)

target.cpufreq.set_all_governors('performance')

logging.info("Target current governor: %s",
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor")
            )
logging.info("Target big CPU max CPUfreq:%s\n\t\t    Target big CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq")
             )

logging.info("Target LITTLE CPU max CPUfreq:%s \n\t\t    Target LITTLE CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu1/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu1/cpufreq/scaling_cur_freq")
             )

04:20:17  INFO    : Target ABI: arm64, CPus: ['A53', 'A72', 'A72', 'A53', 'A53', 'A53']
04:20:18  INFO    : Target current governor: performance
04:20:18  INFO    : Target big CPU max CPUfreq:950000
		    Target big CPU current CPUfreq: 950000
04:20:18  INFO    : Target LITTLE CPU max CPUfreq:1200000 
		    Target LITTLE CPU current CPUfreq: 1200000


In [11]:
logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(out_dir=te.res_dir, cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
nrg_report = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

04:20:19  INFO    : #### Setup FTrace
04:20:21  INFO    : #### Start energy sampling
04:20:21  INFO    : #### Start RTApp execution
04:20:21  INFO    : Workload execution START:
04:20:21  INFO    :    /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/simple_00.json 2>&1
04:20:38  INFO    : #### Read energy consumption: /media/build/workspace/lisa/results/arch_and_freq_invariance/energy.json
04:20:38  INFO    : #### Stop FTrace
04:20:38  INFO    : #### Save FTrace: /media/build/workspace/lisa/results/arch_and_freq_invariance/trace.dat
04:20:40  INFO    : #### Save platform description: /media/build/workspace/lisa/results/arch_and_freq_invariance/platform.json


# Collected results

In [12]:
# All data are produced in the output folder defined by the TestEnv module
logging.info('Content of the output folder %s', te.res_dir)
!ls -la {te.res_dir}

04:20:40  INFO    : Content of the output folder /media/build/workspace/lisa/results/arch_and_freq_invariance


total 6104
drwxrwxr-x 2 steven steven    4096 Jul 18 16:20 .
drwxrwxr-x 5 steven steven    4096 Jul 18 16:20 ..
-rw-rw-r-- 1 steven steven      67 Jul 18 16:20 energy.json
-rw-rw-r-- 1 steven steven     307 Jul 18 16:20 output.log
-rw-rw-r-- 1 steven steven    1348 Jul 18 16:20 platform.json
-rw-r--r-- 1 steven steven    3260 Jul 18 16:20 rt-app-task_p50-0.log
-rw-r--r-- 1 steven steven   12560 Jul 18 16:20 rt-app-task_r20-1.log
-rw-r--r-- 1 steven steven    3265 Jul 18 16:20 simple_00.json
-rw-r--r-- 1 steven steven 6205440 Jul 18 16:20 trace.dat


In [13]:
# Inspect the JSON file used to run the application
with open('{}/simple_00.json'.format(te.res_dir), 'r') as fh:
    rtapp_json = json.load(fh, )
logging.info('Generated RTApp JSON file:')
#print json.dumps(rtapp_json, indent=4, sort_keys=True)

04:20:43  INFO    : Generated RTApp JSON file:


In [14]:
# Dump the energy measured for the LITTLE and big clusters
logging.info('Energy: %s', nrg_report.report_file)
print json.dumps(nrg_report.channels, indent=4, sort_keys=True)

04:20:50  INFO    : Energy: /media/build/workspace/lisa/results/arch_and_freq_invariance/energy.json


{
    "LITTLE": 2.173066999999996, 
    "big": 12.697650999999993
}


In [15]:
# Dump the platform descriptor, which could be useful for further analysis
# of the generated results
logging.info('Platform description: %s', plt_file)
#print json.dumps(plt, indent=4, sort_keys=True)

04:20:51  INFO    : Platform description: /media/build/workspace/lisa/results/arch_and_freq_invariance/platform.json


# Trace inspection

In [16]:
# Suport for FTrace events parsing and visualization
import trappy

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

In [17]:
events_to_parse = my_tests_conf['ftrace']['events']

trace = Trace(plt, te.res_dir, events_to_parse)

ftrace = trace.ftrace

trappy.plotter.plot_trace(ftrace, execnames=['task_p50', "task_r20" ])

04:21:09  INFO    : Platform clusters verified to be Frequency coherent


# Workload Execution Round 2

In [19]:
# Set userspace governor with lowest CPUfreq
#
# For Juno big/LITTLE CPUs
#
logging.info("Target ABI: %s, CPus: %s",
             target.abi,
             target.cpuinfo.cpu_names)

target.cpufreq.set_all_governors('userspace')

target.write_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_setspeed", "450000")
target.write_value("/sys/devices/system/cpu/cpu1/cpufreq/scaling_setspeed", "600000")

logging.info("Target current governor: %s",
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor")
            )
logging.info("Target big CPU min CPUfreq:%s\n\t\t    Target big CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_min_freq"),
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq")
             )

logging.info("Target LITTLE CPU min CPUfreq:%s \n\t\t    Target LITTLE CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu1/cpufreq/scaling_min_freq"),
             target.read_value("/sys/devices/system/cpu/cpu1/cpufreq/scaling_cur_freq")
             )

04:35:42  INFO    : Target ABI: arm64, CPus: ['A53', 'A72', 'A72', 'A53', 'A53', 'A53']
04:35:42  INFO    : Target current governor: userspace
04:35:43  INFO    : Target big CPU min CPUfreq:450000
		    Target big CPU current CPUfreq: 450000
04:35:43  INFO    : Target LITTLE CPU min CPUfreq:600000 
		    Target LITTLE CPU current CPUfreq: 600000


In [20]:
logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(out_dir=te.res_dir, cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
nrg_report = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

04:35:48  INFO    : #### Setup FTrace
04:35:50  INFO    : #### Start energy sampling
04:35:50  INFO    : #### Start RTApp execution
04:35:50  INFO    : Workload execution START:
04:35:50  INFO    :    /data/local/tmp/bin/rt-app /data/local/tmp/devlib-target/simple_00.json 2>&1
04:36:37  INFO    : #### Read energy consumption: /media/build/workspace/lisa/results/arch_and_freq_invariance/energy.json
04:36:38  INFO    : #### Stop FTrace
04:36:38  INFO    : #### Save FTrace: /media/build/workspace/lisa/results/arch_and_freq_invariance/trace.dat
04:36:40  INFO    : #### Save platform description: /media/build/workspace/lisa/results/arch_and_freq_invariance/platform.json


In [21]:
# All data are produced in the output folder defined by the TestEnv module
logging.info('Content of the output folder %s', te.res_dir)
!ls -la {te.res_dir}

04:36:40  INFO    : Content of the output folder /media/build/workspace/lisa/results/arch_and_freq_invariance


total 25792
drwxrwxr-x 2 steven steven     4096 Jul 18 16:36 .
drwxrwxr-x 5 steven steven     4096 Jul 18 16:20 ..
-rw-rw-r-- 1 steven steven       66 Jul 18 16:36 energy.json
-rw-rw-r-- 1 steven steven      307 Jul 18 16:36 output.log
-rw-rw-r-- 1 steven steven     1348 Jul 18 16:36 platform.json
-rw-r--r-- 1 steven steven     3260 Jul 18 16:36 rt-app-task_p50-0.log
-rw-r--r-- 1 steven steven    12560 Jul 18 16:36 rt-app-task_r20-1.log
-rw-r--r-- 1 steven steven     3265 Jul 18 16:36 simple_00.json
-rw-r--r-- 1 steven steven 10186752 Jul 18 16:36 trace.dat
-rw-rw-r-- 1 steven steven  8219685 Jul 18 16:20 trace.raw.txt
-rw-rw-r-- 1 steven steven  7955256 Jul 18 16:20 trace.txt


# Collected Results

# Trace inspection

In [22]:
# Suport for FTrace events parsing and visualization
import trappy

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

In [23]:
events_to_parse = my_tests_conf['ftrace']['events']

trace = Trace(plt, te.res_dir, events_to_parse)

ftrace = trace.ftrace

trappy.plotter.plot_trace(ftrace, execnames=['task_p50', "task_r20" ])

04:37:04  INFO    : Platform clusters verified to be Frequency coherent
